In [1]:
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [2]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")
query = '''
SELECT 
    p.patient_code,
    b.first_name,
    b.last_name,
    b.dob,
    dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    dgs.date AS session_date,
    dg.name AS group_name,
    dh.name AS hub_name,
    lc.name AS hub_commune,
    ld.name AS hub_departement,
    dsd.f_komin AS interview_commune,
    ldt.name AS topic,
    au.email as created_by,
    au.username,
    dgs.created_at as date_session_were_entered
FROM
    caris_db.dream_group_attendance dga
        LEFT JOIN
    dream_member dm ON dm.id_patient = dga.id_patient
        LEFT JOIN
    dreams_surveys_data dsd ON dsd.case_id = dm.case_id
        LEFT JOIN
    dream_group_session dgs ON dgs.id = dga.id_group_session
        LEFT JOIN
    dream_group dg ON dg.id = dgs.id_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
        LEFT JOIN
    lookup_dreams_topic ldt ON ldt.id = dgs.topic
        LEFT JOIN
    patient p ON p.id = dga.id_patient
        LEFT JOIN
    beneficiary b ON b.id_patient = dga.id_patient
        left join 
            auth_users au  on au.id=dgs.created_by
WHERE
    dga.value = 'P'
'''

evaluation_performance_attendance = pd.read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()

In [9]:
evaluation_performance_attendance.shape

(331721, 15)

## Duplication aspect

In [4]:
evaluation_performance_attendance.dtypes

patient_code                         object
first_name                           object
last_name                            object
dob                                  object
interview_date                       object
session_date                         object
group_name                           object
hub_name                             object
hub_commune                          object
hub_departement                      object
interview_commune                    object
topic                                object
created_by                           object
username                             object
date_session_were_entered    datetime64[ns]
dtype: object

In [5]:
epa_deduplicate = evaluation_performance_attendance.drop_duplicates(subset=['patient_code'])

In [8]:
epa_deduplicate.shape

(19022, 15)

## Period of control aspect

In [11]:
 epa_startmars_endnow= epa_deduplicate[epa_deduplicate['date_session_were_entered']>='2021-03-01']

In [12]:
epa_startmars_endnow.shape

(961, 15)

## Count by agents in descending order

In [23]:
performance_session_global  = epa_startmars_endnow.groupby(['created_by']).count()

In [26]:
performance_session_global.sort_values(ascending=False, by='patient_code',inplace=True)

In [27]:
performance_session  = pd.DataFrame( {"nombre_de_filles": performance_session_global['patient_code']})

In [28]:
performance_session

,nombre_de_filles
created_by,
byronfabrice9@gmail.com,298
billyjolicoeur95@gmail.com,211
ernst.belabre@carisfoundationintl.org,135
elionafredly7@gmail.com,73
jhonip77@yahoo.fr,54
francenelstjean.saed@yahoo.com,50
diegoarmando.dorlean@carisfoundationintl.org,49
gerald.poteau@carisfoundationintl.org,46
roselord.despeignes@carisfoundationintl.org,45


## Agent per Commune

In [197]:
performance_agents_par_commune = epa_startmars_endnow.pivot_table(index='created_by',columns='hub_commune',values=['patient_code'],aggfunc='count',fill_value=0,margins=True, margins_name="Grand Total")

In [199]:
performance_agents_par_commune.sort_values(by=('patient_code',"Grand Total") ,ascending=False,inplace=True)

In [200]:
performance_agents_par_commune

patient_code            \
hub_commune                                   Cap-Haïtien Carrefour   
created_by                                                            
Grand Total                                            22        37   
byronfabrice9@gmail.com                                 0        15   
billyjolicoeur95@gmail.com                             22         0   
ernst.belabre@carisfoundationintl.org                   0         0   
elionafredly7@gmail.com                                 0        22   
jhonip77@yahoo.fr                                       0         0   
francenelstjean.saed@yahoo.com                          0         0   
diegoarmando.dorlean@carisfoundationintl.org            0         0   
gerald.poteau@carisfoundationintl.org                   0         0   
roselord.despeignes@carisfoundationintl.org             0         0   

                                                                       \
hub_commune                                  Dessalines Grande Saline   
created_by                                                              
Grand Total                                          50             9   
byronfabrice9@gmail.com                               0             0   
billyjolicoeur95@gmail.com                            0             0   
ernst.belabre@carisfoundationintl.org                 0             0   
elionafredly7@gmail.com                               0             0   
jhonip77@yahoo.fr                                     0             0   
francenelstjean.saed@yahoo.com                       50             0   
diegoarmando.dorlean@carisfoundationintl.org          0             8   
gerald.poteau@carisfoundationintl.org                 0             1   
roselord.despeignes@carisfoundationintl.org           0             0   

                                                                            \
hub_commune                                  Gressier Kenscoff La Chapelle   
created_by                                                                   
Grand Total                                        65      423          43   
byronfabrice9@gmail.com                            65      195           0   
billyjolicoeur95@gmail.com                          0      127           0   
ernst.belabre@carisfoundationintl.org               0        0          43   
elionafredly7@gmail.com                             0       47           0   
jhonip77@yahoo.fr                                   0       54           0   
francenelstjean.saed@yahoo.com                      0        0           0   
diegoarmando.dorlean@carisfoundationintl.org        0        0           0   
gerald.poteau@carisfoundationintl.org               0        0           0   
roselord.despeignes@carisfoundationintl.org         0        0           0   

                                                                             \
hub_commune                                  Petite Rivière de l'Artibonite   
created_by                                                                    
Grand Total                                                              98   
byronfabrice9@gmail.com                                                   0   
billyjolicoeur95@gmail.com                                                3   
ernst.belabre@carisfoundationintl.org                                     0   
elionafredly7@gmail.com                                                   0   
jhonip77@yahoo.fr                                                         0   
francenelstjean.saed@yahoo.com                                            0   
diegoarmando.dorlean@carisfoundationintl.org                             41   
gerald.poteau@carisfoundationintl.org                                    20   
roselord.despeignes@carisfoundationintl.org                              34   

                                                                         \
hub_commune                                  Port-au-

## Reporting Performance de mars 2021 a date

In [201]:
pac = pd.ExcelWriter('rapport_agents_saisie_des_sessions.xlsx',engine='xlsxwriter')
performance_session.to_excel(pac,sheet_name='filles_saisies_global')
performance_agents_par_commune.to_excel(pac,sheet_name="filles_saisies_par_communes")
pac.save()